In [713]:
"""
Image Stitching Problem
(Due date: Nov. 26, 11:59 P.M., 2021)

The goal of this task is to stitch two images of overlap into one image.
You are given 'left.jpg' and 'right.jpg' for your image stitching code testing. 
Note that different left/right images might be used when grading your code. 

To this end, you need to find keypoints (points of interest) in the given left and right images.
Then, use proper feature descriptors to extract features for these keypoints. 
Next, you should match the keypoints in both images using the feature distance via KNN (k=2); 
cross-checking and ratio test might be helpful for feature matching. 
After this, you need to implement RANSAC algorithm to estimate homography matrix. 
(If you want to make your result reproducible, you can try and set fixed random seed)
At last, you can make a panorama, warp one image and stitch it to another one using the homography transform.
Note that your final panorama image should NOT be cropped or missing any region of left/right image. 

Do NOT modify the code provided to you.
You are allowed use APIs provided by numpy and opencv, except “cv2.findHomography()” and
APIs that have “stitch”, “Stitch”, “match” or “Match” in their names, e.g., “cv2.BFMatcher()” and
“cv2.Stitcher.create()”.
If you intend to use SIFT feature, make sure your OpenCV version is 3.4.2.17, see project2.pdf for details.
"""

import cv2
import numpy as np
# np.random.seed(<int>) # you can use this line to set the fixed random seed if you are using np.random
import random
# random.seed(<int>) # you can use this line to set the fixed random seed if you are using random


def solution(left_img, right_img):
    """
    :param left_img:
    :param right_img:
    :return: you need to return the result panorama image which is stitched by left_img and right_img
    """
    sift = cv2.xfeatures2d.SIFT_create() 
    kp_left, des_left = sift.detectAndCompute(left_img, None)
    kp_right, des_right = sift.detectAndCompute(right_img, None)
    all_points = [0]*len(kp_left)*2
    dist=[0]*len(kp_left)*2
    
    #Calculate distance KNN=2
    for i in range(len(kp_left)):
        distance_array = distance_norm(kp_left[i],kp_right,des_left[i],des_right)
        arg = np.argpartition(distance_array,2)
        all_points[2*i]=arg[0]
        all_points[2*i+1] = arg[1]
        dist[2*i] = distance_array[all_points[2*i]]
        dist[2*i+1] = distance_array[all_points[2*i+1]]

    
    # Ratio Testing
    i=0
    thresh = 0.75
    new_points=[]
    while(i<len(all_points)-1):
        if(dist[i]/dist[i+1]<thresh):
            new_points.append((int(i/2),all_points[i]))
        i=i+2
    
    inlier,final_h = Ransac(new_points)
    result_img = offset_warp_stitch(right_img,left_img,final_h)
    
    return result_img


def distance_norm(left,right,x,y):
    return np.sqrt(np.sum(np.square(x - y),axis=1))


def homography_estimate(points,kp_left,kp_right):
    matrix=np.zeros([len(points)*2,9])
    for i in range(len(points)):
        x_left,y_left = kp_left[points[i][0]].pt
        x_right,y_right = kp_right[points[i][1]].pt
        
        j=2*i
        matrix[j]=[x_right,y_right,1,0,0,0,-x_left*x_right,-x_left*y_right,-x_left]
        matrix[j+1]=[0,0,0,x_right,y_right,1,-y_left*x_right,-y_right*y_left,-y_left]
    #print(matrix)
    U, D, V = np.linalg.svd(matrix)
    return V[8]


def Ransac(points,kp_left):
    inlier_max = 0
    final_h = []
    check=[]
    for i in range(1000):
        sample_point = random.sample(points, 4)
        h_matrix = homography_estimate(sample_point)
        h_matrix = (1/h_matrix[8]) * h_matrix
        h_matrix = h_matrix.reshape(3,3)
        temp_inlier = []
        inlier = 0
        for j in range(len(points)):
            
            
            x_right,y_right = kp_right[points[j][1]].pt
            x_left,y_left = kp_left[points[j][0]].pt
            arr = np.array([x_right,y_right,1])
            
            a = np.dot(h_matrix,arr)
            #right = np.transpose(np.matrix([x_right, y_right, 1]))
            #left_cal = np.dot(h_matrix, right)
            #left_cal = (1/left_cal.item(2))*left_cal
            x_left_cal = a[0]/a[2]
            y_left_cal = a[1]/a[2]
            #left = np.transpose(np.matrix([x_left, y_left, 1]))
            #error = left - left_cal
            #error = np.linalg.norm(error)
            
            
            if(np.sqrt((np.square(x_left-x_left_cal)+np.square(y_left-y_left_cal))) < 1 ):
            #if(error<2):   
                #temp_inlier.append(points[j])
                inlier = inlier+1
               
        if(inlier > inlier_max):
            #inlier_max = temp_inlier
            #check = sample_point
            inlier_max = inlier
            final_h = h_matrix
    
    
    return inlier_max,final_h




def offset_warp_stitch(right,left,h):
    (right_width , right_height) = right.shape[0:2]
    (left_width, left_height) = left.shape[0:2]
    
    right_boundary = np.array([[0,0],[0,right_width], [right_height,right_width], [right_height,0]])
    boundary_transform = cv2.perspectiveTransform(np.float32(right_boundary.reshape(-1,1,2)), h)
    left_boundary = np.array([[0,0],[0,left_width], [left_height,left_width], [left_height,0]]) 
    boundary_new = boundary_transform.reshape(4,2)
    all_boundary_x = np.concatenate((left_boundary[:,0] , boundary_new[:,0]))
    x_min = int(min(all_boundary_x))
    x_max = int(max(all_boundary_x))
    all_boundary_y = np.concatenate((left_boundary[:,1] , boundary_new[:,1]))
    y_min = int(min(all_boundary_y))
    y_max = int(max(all_boundary_y))
    translation_matrix = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) 
    print(translation_matrix.dot(h))
    print((x_max-x_min,y_max-y_min))
    result_img = cv2.warpPerspective(right, np.matmul(translation_matrix,h), (x_max-x_min,y_max-y_min))
    result_img[-y_min:right_width - y_min, -x_min:right_height - x_min] = left_img
    
    return result_img
if __name__ == "__main__":
    left_img = cv2.imread('left.jpg')
    right_img = cv2.imread('right.jpg')
    result_img = solution(left_img, right_img)
    cv2.imwrite('results/task1_result.jpg', result_img)


[[ 8.02511298e-01  1.29602266e-02  4.62059798e+02]
 [-6.92095466e-02  9.53476884e-01  6.94501271e+01]
 [-1.99653471e-04  1.36194047e-05  1.00000000e+00]]
(1591, 890)


In [99]:
def distance_norm(left,right,x,y):
    return np.sqrt(np.sum(np.square(x - y),axis=1))
    #sorted_tuple = sorted(point_tuple_array , key = lambda x: x[2])
        

    
    
    
    

In [5]:
!pip install opencv-contrib-python==3.4.2.17

In [8]:
sift_left = cv2.xfeatures2d.SIFT_create()
kp_left = sift_left.detect(left_img, None)
surf_left = cv2.xfeatures2d.SURF_create()
kp_left, des_left = surf_left.compute(left_img, kp_left)


In [9]:
sift_right = cv2.xfeatures2d.SIFT_create()
kp_right = sift_right.detect(right_img, None)
surf_right = cv2.xfeatures2d.SURF_create()
kp_right, des_right = surf_right.compute(right_img, kp_right)

In [269]:
sift = cv2.xfeatures2d.SIFT_create() 
kp_left, des_left = sift.detectAndCompute(left_img, None)
kp_right, des_right = sift.detectAndCompute(right_img, None)

In [270]:
len(kp_left)

4687

In [271]:
all_points = [0]*len(kp_left)*2
dist=[0]*len(kp_left)*2
for i in range(len(kp_left)):
    distance_array = distance_norm(kp_left[i],kp_right,des_left[i],des_right)
    arg = np.argpartition(distance_array,2)
    all_points[2*i]=arg[0]
    all_points[2*i+1] = arg[1]
    dist[2*i] = distance_array[all_points[2*i]]
    dist[2*i+1] = distance_array[all_points[2*i+1]]

    
    #all_points.append(points_sorted[0:2])

    
    
    

In [334]:
i=0
thresh = 0.75
new_points=[]
while(i<len(all_points)-1):
    if(dist[i]/dist[i+1]<thresh):
        new_points.append((int(i/2),all_points[i]))
    i=i+2

    
    

In [539]:
def homography_estimate(points):
    matrix=np.zeros([len(points)*2,9])
    for i in range(len(points)):
        x_left,y_left = kp_left[points[i][0]].pt
        x_right,y_right = kp_right[points[i][1]].pt
        
        j=2*i
        matrix[j]=[x_right,y_right,1,0,0,0,-x_left*x_right,-x_left*y_right,-x_left]
        matrix[j+1]=[0,0,0,x_right,y_right,1,-y_left*x_right,-y_right*y_left,-y_left]
    #print(matrix)
    U, D, V = np.linalg.svd(matrix)
    return V[8]

In [715]:
def Ransac(points):
    inlier_max = 0
    final_h = []
    check=[]
    in_l = []
    for i in range(1000):
        sample_point = random.sample(points, 4)
        h_matrix = homography_estimate(sample_point)
        h_matrix = (1/h_matrix[8]) * h_matrix
        h_matrix = h_matrix.reshape(3,3)
        temp_inlier = []
        inlier = 0
        for j in range(len(points)):
            
            
            x_right,y_right = kp_right[points[j][1]].pt
            x_left,y_left = kp_left[points[j][0]].pt
            #arr = np.array([x_right,y_right,1])
            
            #a = np.dot(h_matrix,arr)
            right = np.transpose(np.matrix([x_right, y_right, 1]))
            left_cal = np.dot(h_matrix, right)
            left_cal = (1/left_cal.item(2))*left_cal
            #x_left_cal = a[0]/a[2]
            #y_left_cal = a[1]/a[2]
            left = np.transpose(np.matrix([x_left, y_left, 1]))
            error = left - left_cal
            error = np.linalg.norm(error)
            
            
            #if(np.sqrt((np.square(x_left-x_left_cal)+np.square(y_left-y_left_cal))) < 5 ):
            if(error<1):   
                temp_inlier.append(points[j])
                inlier = inlier+1
               
        if(inlier > inlier_max):
            #inlier_max = temp_inlier
            #check = sample_point
            inlier_max = inlier
            final_h = h_matrix
            in_l = temp_inlier
    
    return in_l,inlier_max,final_h
            
            
            
            
    
    
    
    

In [716]:
x,inlier,final_h = Ransac(new_points)

In [588]:
final_h

array([[ 7.94209528e-01,  9.84091905e-03,  4.61869999e+02],
       [-5.70811477e-02,  9.44937270e-01, -1.63634112e+00],
       [-2.05360486e-04,  6.99959225e-06,  1.00000000e+00]])

In [589]:
width = left_img.shape[1] + right_img.shape[1]
height = left_img.shape[0] 

output_image_new = cv2.warpPerspective(right_img,h,(2016,756))

result_left = cv2.warpPerspective(left_img, h, (left_img.shape[1], left_img.shape[0]))

output_image_new[:756,:left_img.shape[1]] = result_left




In [590]:
cv2.imwrite('results/task1_result.jpg', output_image_new)

True

In [591]:
    left_height = right_img.shape[1]
    left_width = right_img.shape[0]
    right_height = left_img.shape[1]
    right_width = right_img.shape[0]

    image_frame1 = np.float32([[0, 0], [0, left_width], [left_height, left_width], [left_height, 0]]).reshape(-1, 1, 2)
    image_frame2 = np.float32([[0, 0], [0, right_width], [right_height, right_width], [right_height, 0]]).reshape(-1, 1, 2)
    image_frame2_transformed = cv2.perspectiveTransform(image_frame2, final_h)
    final_image_frame = np.vstack((image_frame1, image_frame2_transformed))
    [minx, miny] = np.int32(final_image_frame.min(axis=0).flatten())
    [maxx, maxy] = np.int32(final_image_frame.max(axis=0).flatten())
    translation_dist = [-minx, -miny]
    h_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])
    anchor_img = cv2.warpPerspective(left_img, h_translation.dot(final_h), (maxx - minx, maxy - miny))
    anchor_img[translation_dist[1]:left_width + translation_dist[1], translation_dist[0]:left_height + translation_dist[0]] = right_img

In [707]:
cv2.imwrite('results/task1_result.jpg', result)

True

In [701]:
def warpPerspective(img1_, img2_, final_H):
    img1_height = img2_.shape[1]
    img1_width = img2_.shape[0]
    img2_height = img1_.shape[1]
    img2_width = img1_.shape[0]

    image_frame1 = np.float32([[0, 0], [0, img1_width], [img1_height, img1_width], [img1_height, 0]]).reshape(-1, 1, 2)
    #print(type(image_frame1))
    image_frame2 = np.float32([[0, 0], [0, img2_width], [img2_height, img2_width], [img2_height, 0]]).reshape(-1, 1, 2)
    
    image_frame2_transformed = cv2.perspectiveTransform(image_frame2, final_H)
    #print(image_frame2_transformed)
    final_image_frame = np.vstack((image_frame1, image_frame2_transformed))
    #print(final_image_frame)
    [minx, miny] = np.int32(final_image_frame.min(axis=0).flatten())
    [maxx, maxy] = np.int32(final_image_frame.max(axis=0).flatten())
    translation_dist = [-minx, -miny]
    h_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])
    print(h_translation.dot(final_H))
    anchor_img = cv2.warpPerspective(img1_, h_translation.dot(final_H), (maxx - minx, maxy - miny))
    print((maxx - minx, maxy - miny))
    anchor_img[translation_dist[1]:img1_width + translation_dist[1], translation_dist[0]:img1_height + translation_dist[0]] = img2_

    return anchor_img

In [702]:
result = warpPerspective(right_img,left_img,final_h)

[[ 7.94209528e-01  9.84091905e-03  4.61869999e+02]
 [-7.22778237e-02  9.45455240e-01  7.23636589e+01]
 [-2.05360486e-04  6.99959225e-06  1.00000000e+00]]
(1591, 894)


In [725]:
cv2.imwrite('results/task1_result.jpg', result)

True

In [611]:
def wrap(img_1,img_2,H):
    dst = cv2.warpPerspective(img_1, H, ((img_1.shape[1] + img_2.shape[1]), img_2.shape[0])) #wraped image
    #dst[0:img_2.shape[0], 0:img_2.shape[1]] = img_2
    #dst[0:img_1.shape[0], 0:img_1.shape[1]] = img_1
    return dst

In [612]:
result = wrap(right_img,left_img,final_h)

In [709]:
def offset_perspective(right,left,h):
    (right_width , right_height) = right.shape[0:2]
    (left_width, left_height) = left.shape[0:2]
    
    right_boundary = np.array([[0,0],[0,right_width], [right_height,right_width], [right_height,0]])
    boundary_transform = cv2.perspectiveTransform(np.float32(right_boundary.reshape(-1,1,2)), h)
    left_boundary = np.array([[0,0],[0,left_width], [left_height,left_width], [left_height,0]]) 
    boundary_new = boundary_transform.reshape(4,2)
    all_boundary_x = np.concatenate((left_boundary[:,0] , boundary_new[:,0]))
    x_min = int(min(all_boundary_x))
    x_max = int(max(all_boundary_x))
    all_boundary_y = np.concatenate((left_boundary[:,1] , boundary_new[:,1]))
    y_min = int(min(all_boundary_y))
    y_max = int(max(all_boundary_y))
    translation_matrix = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) 
    print(translation_matrix.dot(h))
    print((x_max-x_min,y_max-y_min))
    result_img = cv2.warpPerspective(right, np.matmul(translation_matrix,h), (x_max-x_min,y_max-y_min))
    result_img[-y_min:right_width - y_min, -x_min:right_height - x_min] = left_img
    
    return result_img

In [724]:
result = offset_perspective(right_img,left_img,h.reshape(3,3))

[[ 8.00380355e-01  1.38103126e-02  4.61836922e+02]
 [-7.26851626e-02  9.54497718e-01  7.25380946e+01]
 [-2.05555120e-04  1.63602919e-05  1.00000000e+00]]
(1600, 895)


In [628]:
right_img.shape

(756, 1008, 3)

In [717]:
x

[(2317, 4),
 (2328, 10),
 (2342, 14),
 (2346, 33),
 (2347, 30),
 (2350, 29),
 (2355, 34),
 (2359, 36),
 (2363, 49),
 (2364, 43),
 (2374, 63),
 (2376, 60),
 (2377, 69),
 (2378, 61),
 (2381, 70),
 (2387, 78),
 (2388, 85),
 (2389, 86),
 (2390, 91),
 (2394, 84),
 (2406, 94),
 (2407, 100),
 (2408, 101),
 (2409, 102),
 (2418, 110),
 (2422, 121),
 (2432, 125),
 (2436, 129),
 (2441, 133),
 (2443, 140),
 (2444, 130),
 (2447, 137),
 (2448, 143),
 (2449, 149),
 (2454, 144),
 (2455, 148),
 (2457, 151),
 (2458, 152),
 (2460, 153),
 (2463, 168),
 (2468, 167),
 (2470, 170),
 (2471, 179),
 (2473, 173),
 (2474, 180),
 (2477, 183),
 (2480, 185),
 (2481, 196),
 (2488, 202),
 (2491, 199),
 (2494, 200),
 (2501, 211),
 (2503, 224),
 (2508, 225),
 (2509, 226),
 (2510, 228),
 (2511, 227),
 (2516, 236),
 (2520, 239),
 (2524, 241),
 (2525, 252),
 (2530, 254),
 (2542, 271),
 (2556, 299),
 (2564, 306),
 (2569, 308),
 (2570, 316),
 (2573, 319),
 (2578, 322),
 (2579, 321),
 (2598, 333),
 (2601, 341),
 (2603, 347),


In [718]:
h = homography_estimate(x)

In [723]:
h = h/h[8]